In [16]:
import numpy as np
import pandas as pd
import random
from pprint import pprint

# Read in CSV dataset

In [17]:
# define data with pandas dataframe
# mushroom dataframe
# col_names = ['edibility','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
# # load dataset
# df = pd.read_csv('agaricus-lepiota.data.csv', header=0, names=col_names)
# df['class'] = df.edibility
# df = df.drop(columns = 'edibility')

#dropped due to missing
#df = df.drop(columns = "stalk-root")


#make sure last column is class


#credit card dataframe
col_names = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','class']
df = pd.read_csv('crx.data.csv', header = 0, names = col_names)


# Train/Test Split

In [18]:
#testSize = proportion or exact # of rows
#didn't use k-folds or anything fancy, just random splits

In [19]:
def trainTestSplit(df, testSize = 0.2):
    #default 80/20 split, seemed like a good amount
    
    if isinstance(testSize, float):
        testSize = round(testSize * len(df))

    indices = df.index.tolist()
    #convert indices to list to choose random test indices
    testIndices = random.sample(population = indices, k = testSize)

    #slice to get test
    test = df.loc[testIndices]
    #drop test from train
    train = df.drop(testIndices)
    return train, test

In [20]:
#split the data
train, test = trainTestSplit(df)

# Helper functions

### Helper: Check Purity

##### using this instead of entropy == 0

In [21]:
def checkPurity(data):
    classColumn = data[:, -1]
    uniqueClasses = np.unique(classColumn)
    
    if len(uniqueClasses) == 1:
        return True
    else:
        return False

### Helper: Classify

###### returns class that appears most in data

In [22]:
def classifyData(data):
    classColumn = data[:, -1]
    uniqueClasses, countsUniqueClasses = np.unique(classColumn, returnCounts=True)
    #unique values and times they appear

    
    index = countsUniqueClasses.argmax()
    classification = uniqueClasses[index]
    #chooses classification that appears most often
    
    return classification

### Helper: categorical vs continuous feature

In [23]:
def catOrCon(df):
    featureTypes = []
    numUniqueValuesThreshold = 15
    
    for column in df.columns:
        uniqueValues = df[column].unique()
        exampleValue = uniqueValues[0]
        
        if (isinstance(exampleValue, str)) or (len(uniqueValues) <= numUniqueValuesThreshold):
            featureTypes.append("categorical")
        else:
            featureTypes.append("continuous")
    
    
    return featureTypes

### Helpers: Splits

##### identifies splits for continuous data

In [24]:
def allPossibleSplits(data):
    
    potentialSplits = {}
    _, numColumns = data.shape
    for columnIndex in range(numColumns - 1):
        values = data[:,columnIndex]
        uniqueValues = np.unique(values)
    
        #for continuous
        typeOfFeature = FEATURE_TYPES[splitColumn]
        
        if typeOfFeature == "continuous":
            potentialSplits[columnIndex] = []
            for index in range(len(uniqueValues)):
                if index != 0:
                    currentValue = uniqueValues[index]
                    previousValue = uniqueValues[index-1]
                    potentialSplit = (currentValue + previousValue) / 2

                    potentialSplits[columnIndex].append(potentialSplit)
        
        #for categorical
        elif len(uniqueValues) > 1:
            potentialSplits[columnIndex] = uniqueValues 
    
    return potentialSplits



def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    
    typeOfFeature = FEATURE_TYPES[splitColumn]
    if typeOfFeature == "continuous":
        dataBelow = data[splitColumnValues <= splitValue]
        dataAbove = data[splitColumnValues > splitValue]
        
    else:
        dataBelow = data[splitColumnValues == splitValue]
        dataAbove = data[splitColumnValues != splitValue]
        
    return dataBelow, dataAbove

### Helpers: Entropy/Info Gain

In [25]:
def calculateEntropy(data):
    classColumn = data[:, -1]
    _, counts = np.unique(classColumn, returnCounts = True)
    probPos = counts / counts.sum()
    entropy = sum( probPos * -np.log2(probPos) )
    
    return entropy


def calculateInfoGain(dataBelow, dataAbove):
    nDataPoints = len(dataBelow) + len(dataAbove)

    pDataBelow = len(dataBelow) / nDataPoints
    pDataAbove = len(dataAbove) / nDataPoints

    infoGain = (pDataBelow * calculateEntropy(dataBelow)
                      + pDataAbove * calculateEntropy(dataAbove))

    
    return infoGain


def determineBestSplit(data, potentialSplits):
    infoGain = 999
    for columnIndex in potentialSplits:
        for value in potentialSplits[columnIndex]:
            dataBelow, dataAbove = splitData(data, splitColumn=columnIndex, splitValue = value)
            currentInfoGain = calculateInfoGain(dataBelow, dataAbove)

            if currentInfoGain <= infoGain:
                infoGain = currentInfoGain
                bestSplitColumn = columnIndex
                bestSplitValue = value

    return bestSplitColumn, bestSplitValue

# Main

In [26]:
def decisionTreeAlgorithm(df, counter = 0, minSamples = 2, maxDepth = 999):
    
    #needed to use global variables and store on first call
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = catOrCon(df)
        
        data = df.values
    else:
        data = df
        
    #base case
    if (checkPurity(data)) or (len(data) < minSamples) or (counter == maxDepth):
        classification = classifyData(data)
        return classification
    
    #recursive
    #determine the splits and split the data
    #determine continuous or categorical and phrase question correctly
    #assign positive/negative answers
    else:
        counter += 1
        potentialSplits = allPossibleSplits(data)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        
        featureName = COLUMN_HEADERS[splitColumn]
        typeOfFeature = FEATURE_TYPES[splitColumn]
        if typeOfFeature == "continuous":
            question = "{} <= {}".format(featureName, splitValue)
        else:
            question = "{} = {}".format(featureName, splitValue)
        subTree = {question: []}
        
        positiveAnswer = decisionTreeAlgorithm(dataBelow, counter, minSamples, maxDepth)
        negativeAnswer = decisionTreeAlgorithm(dataAbove, counter, minSamples, maxDepth)
        
        if positiveAnswer == negativeAnswer:
            subTree = positiveAnswer
        else:
            subTree[question].append(positiveAnswer)
            subTree[question].append(negativeAnswer)

        return subTree
    

In [ ]:
tree = decisionTreeAlgorithm(train)
pprint(tree)

# Classification/Accuracy

In [ ]:
def classifyExample(example, tree):
    question = list(tree.keys())[0]
    featureName, comparisonOperator, value = question.split()
    if comparisonOperator == "<=":
        if example[featureName] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    else:
        if str(example[featureName]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    if not isinstance(answer, dict):
        return answer

    else:
        residualTree = answer
        return classifyExample(example, residualTree)

    
    
def calculateAccuracy(df, tree):
    
    df["classification"] = df.apply(classifyExample, axis = 1, args= (tree,))
    df["classification_correct"] = df["classification"] == df["class"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [ ]:
testAccuracy = calculateAccuracy(test, tree)
testAccuracy 

In [ ]:
trainAccuracy = calculateAccuracy(train, tree)
trainAccuracy